## Exercise 4



# a)

If the maximum Degree is N for one variable, than it can be in the polynomial part as {0,1,...,N} variants.
THis means that there are (N_x + 1 ) * (N_Y + 1)  * ( N_z + 1) possible  combinations of these variables resulting in the same amount of coefficients.

Version 1: t * 4 (x,y,z, and coeff)

Version 2:This would result in an three dimensional matrix having the shape ((N_x + 1 ), (N_Y + 1),( N_z + 1))

# b)

In [10]:
#import rec_sys.jax_intro as jax_intro
import jax.numpy as jnp
import jax
import time
import random
rnd_seed = int(time.time())
rnd_key = jax.random.PRNGKey(rnd_seed)

In [11]:


def gen_polynomial(Nx,Ny,Nz,t):
    
    max_arr = [Nx,Ny,Nz]
    
    
    indx = jax.random.randint(  rnd_key, (t,3), jnp.zeros((t,3)),
                                jnp.reshape(jnp.repeat(jnp.array([Nx+1,Ny+1,Nz+1]),repeats=t,axis=0 ),
                                newshape=(3,t)).T)
    
    
    for i in range(3):
        
        if max(indx[:,i])< max_arr[i] :
            rnd_indx = random.randint(0, t-1)
            indx = indx.at[rnd_indx,i].set(max_arr[i]) 
    LIM = 5
    rnd_coeff = jax.random.randint(  rnd_key, (t,1), -LIM * jnp.ones((t,1)),
                                    LIM * jnp.ones((t,1))
                                     )
    coeff_mat = jnp.hstack([indx,rnd_coeff])
    return(coeff_mat)



def f(x,y,z,polym):
    res = jnp.zeros(x.shape[0])
    for i in range(polym.shape[0]):
        res += polym[i,3] * jnp.power(x,polym[i,0])* jnp.power(y,polym[i,1])* jnp.power(z,polym[i,2])
    return(res)




def generate_noisy_set(N,coeff_mat):
    noise_frac, rnd_seed =  0.25, int(time.time())
    x,y,z = jnp.linspace(-3, 3,N),jnp.linspace(-3, 3,N),jnp.linspace(-3, 3,N)
    res_pure = f(x,y,z,coeff_mat)
    # Add some noise to data
    rnd_key = jax.random.PRNGKey(rnd_seed)
    y_with_noise = res_pure + res_pure * noise_frac * jax.random.normal(rnd_key, (N,))
    return x,y,z,y_with_noise

polym = gen_polynomial(3,1,2,5)
print(polym)
#print(generate_noisy_set(10,polym) )

#print(generate_noisy_set(2,jnp.array([[1,1,1,1],[0,0,0,0]])) )



[[ 0  1  2  1]
 [ 2  0  2  1]
 [ 3  0  2  4]
 [ 1  0  1 -3]
 [ 2  0  0 -5]]


In [12]:
def gen_polynomial2(Nx,Ny,Nz,t):
    LIM = 3
    rnd_seed = int(time.time())
    max_arr = [Nx,Ny,Nz]

    coeff_mat = jnp.zeros((Nx+1,Ny+1,Nz+1))
    for _ in range(t):
        rnd_coeff = random.randint(-LIM,LIM)
        while rnd_coeff ==0:
            rnd_coeff = random.randint(-LIM,LIM)
        coeff_mat = coeff_mat.at[random.randint(0,Nx),random.randint(0,Ny),random.randint(0,Nz)].set(rnd_coeff) 


    while not jnp.any(coeff_mat[-1,:,:]):
        coeff_mat = jnp.roll(coeff_mat,1,0)
    while not jnp.any(coeff_mat[:,-1,:]):
        coeff_mat = jnp.roll(coeff_mat,1,1)
    while not jnp.any(coeff_mat[...,-1]):
        coeff_mat = jnp.roll(coeff_mat,1,2)
    return coeff_mat


def f2(x,y,z,polym):
    res = jnp.zeros(x.shape[0])
    for i in range(polym.shape[0]):
        for j in range(polym.shape[1]):
            for k in range(polym.shape[2]):
                res += polym[i,j,k] * jnp.power(x,i)* jnp.power(y,j)* jnp.power(z,k)
    return(res)

def generate_noisy_set2(N,coeff_mat):
    noise_frac, rnd_seed =  0.25, int(time.time())
    x,y,z = jnp.linspace(-3, 3,N),jnp.linspace(-3, 3,N),jnp.linspace(-3, 3,N)
    res_pure = f2(x,y,z,coeff_mat)
    # Add some noise to data

    res_with_noise = res_pure + res_pure * noise_frac * jax.random.normal(rnd_key, (N,))
    return x,y,z,res_with_noise

# c)

In [16]:
 
#polynomials:
polym1 = gen_polynomial(2,4,6,12)
polym2 = gen_polynomial(3,1,2,5)

def SGD(polynomial,Nx,Ny,Nz,t):

    # %% Run the SG loop
    num_epochs = 5
    learning_rate = 0.05
    train_set_size = 10000
    train_ds  = jnp.asarray(generate_noisy_set(train_set_size ,polynomial))
    
    param_w = gen_polynomial(Nx,Ny,Nz,t).astype(float)  # Initial guess for the parameter

    # %% Define a simple loss function and its gradient
    def loss(param_w, data):
        # return  jnp.sum((data[:,1] - f(data[:,0], param_w))**2)
        res = f(data[0],data[1],data[2], param_w)

        return jnp.log(jnp.sum((data[3] - res) ** 2)) 


    # Using JAX automatic differentiation - autograd
    grad_loss = jax.grad(loss)#,allow_int=True)

    num_points_per_batch = train_set_size // 5
    print("\n===== Running Stochastic Gradient Descent =====")
    for epoch in range(num_epochs):
        # Get points for the current batch
        for i in range(0, train_set_size, num_points_per_batch):
            batch = train_ds[:,i:i + num_points_per_batch]
            grad = grad_loss(param_w, batch)
            param_w = param_w - learning_rate * grad

        print(f"Epoch {epoch}: param_w={param_w}, grad={grad}, loss={loss(param_w, train_ds)}")

    print(polynomial)
    print(param_w)
SGD(polym2,3,1 ,2,5)



===== Running Stochastic Gradient Descent =====
Epoch 0: param_w=[[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]], grad=[[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]], loss=nan
Epoch 1: param_w=[[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]], grad=[[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]], loss=nan


KeyboardInterrupt: 

In [18]:
 
#polynomials:
polym1 = gen_polynomial2(2,4,6,12)
polym2 = gen_polynomial2(3,1,2,5)
print(polym2)
def SGD(polynomial,Nx,Ny,Nz,t):

    # %% Run the SG loop
    num_epochs = 50
    learning_rate = 0.05
    train_set_size = 10000
    train_ds  = jnp.asarray(generate_noisy_set2(train_set_size ,polynomial))
    
    param_w = jnp.zeros((Nx+1,Ny+1,Nz+1)) #gen_polynomial2(Nx,Ny,Nz,t).astype(float)  # Initial guess for the parameter

    # %% Define a simple loss function and its gradient
    def loss(param_w, data):
        # return  jnp.sum((data[:,1] - f(data[:,0], param_w))**2)
        res = f2(data[0],data[1],data[2], param_w)
      
        return jnp.log(jnp.sum((data[3] - res) ** 2)) 

    # Using JAX automatic differentiation - autograd
    grad_loss = jax.grad(loss)

    num_points_per_batch = train_set_size // 5
    print("\n===== Running Stochastic Gradient Descent =====")
    for epoch in range(num_epochs):
        # Get points for the current batch
        for i in range(0, train_set_size, num_points_per_batch):
            batch = train_ds[:,i:i + num_points_per_batch]
            grad = grad_loss(param_w, batch)
            param_w = param_w - learning_rate * grad

        print(f"Epoch {epoch}:  loss={loss(param_w, train_ds)}")
    print(polynomial)
    print(param_w)
SGD(polym2,3,1,2,5)


[[[ 0.  0.  0.]
  [ 0.  0.  0.]]

 [[ 0.  3.  0.]
  [ 0.  2.  0.]]

 [[ 0. -3.  0.]
  [ 0.  0.  0.]]

 [[ 0.  0.  0.]
  [ 0.  0.  3.]]]

===== Running Stochastic Gradient Descent =====
Epoch 0:  loss=21.97057342529297
Epoch 1:  loss=21.860944747924805
Epoch 2:  loss=21.738082885742188
Epoch 3:  loss=21.601057052612305
Epoch 4:  loss=21.44331169128418
Epoch 5:  loss=21.2585506439209
Epoch 6:  loss=21.037555694580078
Epoch 7:  loss=20.768587112426758
Epoch 8:  loss=20.44303321838379
Epoch 9:  loss=20.087587356567383
Epoch 10:  loss=19.78936767578125
Epoch 11:  loss=19.600202560424805
Epoch 12:  loss=19.47283172607422
Epoch 13:  loss=19.42186164855957
Epoch 14:  loss=19.374752044677734
Epoch 15:  loss=19.36151885986328
Epoch 16:  loss=19.33759117126465
Epoch 17:  loss=19.330074310302734
Epoch 18:  loss=19.323762893676758
Epoch 19:  loss=19.31654167175293
Epoch 20:  loss=19.31874656677246
Epoch 21:  loss=19.30645751953125
Epoch 22:  loss=19.308258056640625
Epoch 23:  loss=19.29726791381836